In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from pycocotools.coco import COCO
from skimage import io,transform
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)

import cam_net
import cam_utils

In [ ]:
dataDir='data'
dataType='val2017'
annFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)
coco=COCO(annFile)

In [ ]:
yes_files,no_files = cam_utils.split_data(coco)